In [0]:
# some libraries
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving isr_series.csv to isr_series.csv


In [4]:
# to read the dataframe
import io
df = pd.read_csv(io.StringIO(uploaded['isr_series.csv'].decode('utf-8')))
df

,Date,ISR_TM_TAX_MANF_BR_ZS,ISR_TM_TAX_MRCH_BR_ZS,ISR_FM_LBL_BMNY_CN,ISR_FM_LBL_BMNY_GD_ZS,ISR_FM_LBL_BMNY_ZG,ISR_FM_LBL_BMNY_IR_ZS,ISR_IQ_WEF_CUST_XQ,ISR_IC_BUS_DISC_XQ,ISR_GC_BAL_CASH_CN,ISR_GC_BAL_CASH_GD_ZS,ISR_GC_DOD_TOTL_GD_ZS,ISR_GC_DOD_TOTL_CN,ISR_NE_GDI_STKB_KN,ISR_AG_PRD_CREL_MT,ISR_AG_YLD_CREL_KG,ISR_NE_GDI_STKB_CN,ISR_NE_GDI_STKB_CD,ISR_BN_RES_INCL_CD,ISR_NV_MNF_CHEM_ZS_UN,ISR_SE_PRM_UNER_FE,ISR_SE_PRM_UNER,ISR_FS_AST_CGOV_GD_ZS,ISR_SE_PRM_UNER_MA,ISR_FM_AST_CGOV_ZG_M3,ISR_FM_AST_PRVT_ZG_M3,ISR_FS_AST_DOMO_GD_ZS,ISR_FM_AST_DOMO_ZG_M3,ISR_EN_ATM_CO2E_PP_GD_KD,ISR_EN_ATM_CO2E_PP_GD,ISR_EN_ATM_CO2E_KD_GD,ISR_EN_ATM_CO2E_KT,ISR_EN_CO2_ETOT_ZS,ISR_EN_ATM_CO2E_PC,ISR_EN_CO2_ETOT_MT,ISR_EN_ATM_CO2E_GF_ZS,ISR_EN_ATM_CO2E_GF_KT,ISR_EN_CO2_MANF_ZS,ISR_EN_ATM_CO2E_LF_ZS,ISR_EN_ATM_CO2E_LF_KT,...,ISR_NV_AGR_TOTL_ZS,ISR_NV_AGR_TOTL_KD_ZG,ISR_EA_PRD_AGRI_KD,ISR_NV_AGR_TOTL_CN,ISR_NV_AGR_TOTL_KD,ISR_NV_AGR_TOTL_KN,ISR_NV_AGR_TOTL_CD,ISR_IS_AIR_GOOD_MT_K1,ISR_IS_AIR_PSGR,ISR_IS_AIR_DPRT,ISR_EG_USE_COMM_CL_ZS,ISR_ER_H2O_FWAG_ZS,ISR_ER_H2O_FWDM_ZS,ISR_ER_H2O_FWTL_K3,ISR_ER_H2O_FWTL_ZS,ISR_ER_H2O_FWIN_ZS,ISR_AG_LND_ARBL_HA,ISR_AG_LND_ARBL_HA_PC,ISR_AG_LND_ARBL_ZS,ISR_MS_MIL_TOTL_P1,ISR_MS_MIL_TOTL_TF_ZS,ISR_MS_MIL_XPRT_KD,ISR_MS_MIL_MPRT_KD,ISR_FB_ATM_TOTL_P5,ISR_AG_LND_PRCP_MM,ISR_IC_TAX_METG,ISR_IC_CUS_DURS_EX,ISR_FB_BNK_CAPA_ZS,ISR_FD_RES_LIQU_AS_ZS,ISR_VC_BTL_DETH,ISR_FB_AST_NPER_ZS,ISR_TM_TAX_MANF_BC_ZS,ISR_TM_TAX_MRCH_BC_ZS,ISR_TM_TAX_TCOM_BC_ZS,ISR_SH_STA_BRTC_ZS,ISR_FB_CBK_BRWR_P3,ISR_EN_BIR_THRD_NO,ISR_SP_DYN_CBRT_IN,ISR_TM_TAX_TCOM_BR_ZS,CPI
0,1999-12-31,13.43,23.45,4.062758e+11,83.745559,15.538637,4.341603,NaN,NaN,-1.244000e+10,-2.563764,91.425594,4.435340e+11,8.517000e+09,123030.0,1668.4,6.402000e+09,1.546489e+09,9.903337e+08,11.564320,8085.0,15150.0,3.681101,7065.0,1.552017,12.498361,NaN,NaN,0.358229,0.402342,0.346619,55694.396,65.384615,9.092963,32.59,0.032921,18.335,3.037475,50.776929,28279.904,...,1.433363,-5.990033,38697.268547,6.953691e+09,2.399231e+09,8.970723e+09,1.679757e+09,1034.300049,4033200.0,51600.000000,3.225168,NaN,NaN,NaN,NaN,NaN,333000.000000,0.054367,15.388170,179550.0,6.727279,1.900000e+08,1.191000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,69.88,74.32,90.31,NaN,NaN,NaN,21.5,49.97,68.0
1,2000-12-31,13.43,23.45,4.388617e+11,81.332877,8.020643,4.623273,NaN,NaN,-2.100047e+10,-3.890266,NaN,NaN,7.467000e+09,182870.0,2443.3,8.669000e+09,2.126162e+09,8.853507e+08,10.271243,7371.0,13726.0,-0.269958,6355.0,-4.754117,10.720795,NaN,NaN,0.358322,0.384342,0.346709,60259.811,65.693963,9.581779,35.41,0.030427,18.335,2.681014,48.408690,29170.985,...,1.264912,7.968936,42465.963986,6.825302e+09,2.590424e+09,9.685595e+09,1.673976e+09,885.704000,4442967.0,56427.000000,3.280555,NaN,NaN,NaN,NaN,NaN,338000.000000,0.053745,15.619224,180500.0,6.519882,4.040000e+08,3.680000e+08,NaN,NaN,NaN,NaN,7.300000,NaN,77.0,6.900000,69.88,74.32,90.31,NaN,NaN,NaN,21.7,49.97,66.0
2,2001-12-31,13.48,23.48,4.766490e+11,86.709035,8.610294,4.847772,NaN,NaN,-3.326151e+10,-6.048625,NaN,NaN,7.525000e+09,241640.0,2454.8,1.046300e+10,2.487814e+09,-2.112444e+07,10.493848,5075.0,10913.0,5.388487,5838.0,7.081447,7.065718,79.901643,NaN,0.378069,0.396525,0.365816,63596.781,66.464718,9.876810,36.64,0.028830,18.335,2.334284,48.157758,30626.784,...,1.471491,8.535030,46858.620843,8.088946e+09,2.811517e+09,1.051226e+10,1.923329e+09,857.561000,3967140.0,47648.000000,3.275443,NaN,NaN,NaN,NaN,NaN,342000.000000,0.053114,15.804067,171500.0,6.049483,4.690000e+08,1.310000e+08,NaN,NaN,NaN,NaN,7.700000,12.942343,244.0,8.200000,69.88,74.32,90.31,NaN,NaN,NaN,21.2,49.96,76.0
3,2002-12-31,11.30,22.20,5.292942e+11,92.275439,11.044869,4.638868,NaN,NaN,-4.184628e+10,-7.293939,NaN,NaN,2.374000e+09,271105.0,2689.5,4.601000e+09,9.711258e+08,-6.050213e+08,12.656288,6147.0,13169.0,3.990693,7022.0,-1.412021,7.594897,83.763059,8.651860,0.354997,0.361071,0.343493,59812.437,67.983122,9.103872,39.57,0.024523,14.668,2.177215,47.201275,28232.233,...,1.503393,3.987643,50407.422693,8.623500e+

In [0]:
import datetime
def datetiem(x):
    date_time_obj = datetime.datetime.strptime(x, '%Y-%m-%d' )
    return date_time_obj

In [0]:
df["Date"] = df["Date"].apply(lambda x: datetiem(x))
df.index = df.Date # index as date because time is not a variable
df=df.drop(['Date'], axis=1)

In [9]:
# df0 = df.fillna(0)
# removes columns where more than 30% of data is missing
# rationale for 0.7, if we can train test split for 0.7/ 0.3 we can accept at least 70% filled data
# fills remaining missing values with the mean of the column   
df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.80].fillna(df.mean())
# df_mean = df.loc[:, pd.notnull(df).sum()>len(df)*.70].fillna(df.interpolate())
df_mean

,ISR_TM_TAX_MANF_BR_ZS,ISR_TM_TAX_MRCH_BR_ZS,ISR_FM_LBL_BMNY_CN,ISR_FM_LBL_BMNY_GD_ZS,ISR_FM_LBL_BMNY_ZG,ISR_FM_LBL_BMNY_IR_ZS,ISR_GC_BAL_CASH_CN,ISR_GC_BAL_CASH_GD_ZS,ISR_AG_PRD_CREL_MT,ISR_AG_YLD_CREL_KG,ISR_NE_GDI_STKB_CN,ISR_NE_GDI_STKB_CD,ISR_BN_RES_INCL_CD,ISR_SE_PRM_UNER_FE,ISR_SE_PRM_UNER,ISR_FS_AST_CGOV_GD_ZS,ISR_SE_PRM_UNER_MA,ISR_FM_AST_CGOV_ZG_M3,ISR_FM_AST_PRVT_ZG_M3,ISR_FS_AST_DOMO_GD_ZS,ISR_FM_AST_DOMO_ZG_M3,ISR_EN_ATM_CO2E_PP_GD_KD,ISR_EN_ATM_CO2E_PP_GD,ISR_EN_ATM_CO2E_KD_GD,ISR_EN_ATM_CO2E_KT,ISR_EN_CO2_ETOT_ZS,ISR_EN_ATM_CO2E_PC,ISR_EN_ATM_CO2E_GF_ZS,ISR_EN_ATM_CO2E_GF_KT,ISR_EN_CO2_MANF_ZS,ISR_EN_ATM_CO2E_LF_ZS,ISR_EN_ATM_CO2E_LF_KT,ISR_EN_CO2_OTHX_ZS,ISR_EN_CO2_BLDG_ZS,ISR_EN_ATM_CO2E_SF_ZS,ISR_EN_CO2_TRAN_ZS,ISR_EN_ATM_CO2E_EG_ZS,ISR_EN_ATM_CO2E_SF_KT,ISR_EG_USE_CRNW_ZS,ISR_NY_GDP_COAL_RT_ZS,...,ISR_NY_ADJ_NNAT_GN_ZS,ISR_NY_ADJ_NNAT_CD,ISR_NY_ADJ_DFOR_CD,ISR_NY_ADJ_DPEM_GN_ZS,ISR_SP_ADO_TFRT,ISR_NY_ADJ_DPEM_CD,ISR_SP_POP_DPND_OL,ISR_SP_POP_DPND,ISR_SP_POP_DPND_YG,ISR_AG_LND_AGRI_K2,ISR_AG_LND_AGRI_ZS,ISR_TX_VAL_AGRI_ZS_UN,ISR_TM_VAL_AGRI_ZS_UN,ISR_NV_AGR_TOTL_ZS,ISR_NV_AGR_TOTL_KD_ZG,ISR_EA_PRD_AGRI_KD,ISR_NV_AGR_TOTL_CN,ISR_NV_AGR_TOTL_KD,ISR_NV_AGR_TOTL_KN,ISR_NV_AGR_TOTL_CD,ISR_IS_AIR_GOOD_MT_K1,ISR_IS_AIR_PSGR,ISR_IS_AIR_DPRT,ISR_EG_USE_COMM_CL_ZS,ISR_AG_LND_ARBL_HA,ISR_AG_LND_ARBL_HA_PC,ISR_AG_LND_ARBL_ZS,ISR_MS_MIL_TOTL_P1,ISR_MS_MIL_TOTL_TF_ZS,ISR_MS_MIL_XPRT_KD,ISR_MS_MIL_MPRT_KD,ISR_FB_BNK_CAPA_ZS,ISR_FD_RES_LIQU_AS_ZS,ISR_FB_AST_NPER_ZS,ISR_TM_TAX_MANF_BC_ZS,ISR_TM_TAX_MRCH_BC_ZS,ISR_TM_TAX_TCOM_BC_ZS,ISR_SP_DYN_CBRT_IN,ISR_TM_TAX_TCOM_BR_ZS,CPI
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-12-31,13.43,23.45,4.062758e+11,83.745559,15.538637,4.341603,-1.244000e+10,-2.563764,123030.0,1668.4,6.402000e+09,1.546489e+09,9.903337e+08,8085.0,15150.0,3.681101,7065.0,1.552017,12.498361,81.123916,4.736520,0.358229,0.402342,0.346619,55694.396000,65.384615,9.092963,0.032921,18.335000,3.037475,50.776929,28279.904000,3.076923,3.767258,43.501449,24.714004,3.321343,24227.869000,0.052384,0.0,...,8.732203,9.780068e+09,406490.054311,0.124004,17.0126,1.388841e+08,16.099180,61.791909,45.692730,5590.000000,25.831793,1.522071,1.123974,1.433363,-5.990033,38697.268547,6.953691e+09,2.399231e+09,8.970723e+09,1.679757e+09,1034.300049,4033200.0,51600.000000,3.225168,333000.000000,0.054367,15.388170,179550.0,6.727279,1.900000e+08,1.191000e+09,6.418794,13.911778,9.000000,69.88,74.32,90.31,21.5,49.97,68.0
2000-12-31,13.43,23.45,4.388617e+11,81.332877,8.020643,4.623273,-2.100047e+10,-3.890266,182870.0,2443.3,8.669000e+09,2.126162e+09,8.853507e+08,7371.0,13726.0,-0.269958,6355.0,-4.754117,10.720795,81.123916,4.736520,0.358322,0.384342,0.346709,60259.811000,65.693963,9.581779,0.030427,18.335000,2.681014,48.408690,29170.985000,2.972825,4.285975,46.838678,24.366223,3.304715,28224.899000,0.048172,0.0,...,8.527040,1.057353e+10,0.000000,0.120043,16.6924,1.488528e+08,16.175080,61.553005,45.377925,5660.000000,26.155268,0.946455,1.021487,1.264912,7.968936,42465.963986,6.825302e+09,2.590424e+09,9.685595e+09,1.673976e+09,885.704000,4442967.0,56427.000000,3.280555,338000.000000,0.053745,15.619224,180500.0,6.519882,4.040000e+08,3.680000e+08,7.300000,13.911778,6.900000,69.88,74.32,90.31,21.7,49.97,66.0
2001-12-31,13.48,23.48,4.766490e+11,86.709035,8.610294,4.847772,-3.326151e+10,-6.048625,241640.0,2454.8,1.046300e+10,2.487814e+09,-2.112444e+07,5075.0,10913.0,5.388487,5838.0,7.081447,7.065718,79.901643,4.736520,0.378069,0.396525,0.365816,63596.781000,66.464718,9.876810,0.028830,18.335000,2.334284,48.157758,30626.784000,3.795438,3.741981,48.128928,23.645759,3.322122,30608.449000,0.045885,0.0,...,7.329594,9.161992e+09,0.000000,0.120631,16.3722,1.507885e+08,16.168316,61.474764,45.306448,5610.000000,25.924214,0.869360,0.965968,1.471491,8.535030,46858.620843,8.088946e+09,2.811517e+09,1.051226e+10,1.923329e+09,857.561000,3967140.0,47648.000000,3.275443,342000.000000,0.053114,15.804067,171500.0,6.049483,4.69

In [0]:
CPI_list = list(df_mean.CPI)
def pct_df(df):
    df.drop(["CPI"], axis=1)
    new_df = df.pct_change()
    new_df.replace(np.nan, 0)
    new_df = new_df.replace([np.inf, -np.inf], 0)
    new_df = new_df.dropna(how='all')
    new_df['CPI']=CPI_list[1:]
    new_df = new_df.fillna(df.mean())
    return new_df
def UpsampleByTimeIndex(df,bytime,method):
    upsampled = df.resample(bytime)
    interpolated = upsampled.interpolate(method=method)
    return interpolated
# Ignore the index and treat the values as equally space
# ‘nearest’, ‘zero’, ‘slinear’, ‘quadratic’, ‘cubic’, ‘spline’, ‘barycentric’, ‘polynomial’
# .agg aggregates/ groups by specified time frame
# D for days, M for months
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.resample.Resampler.aggregate.html 
# https://machinelearningmastery.com/resample-interpolate-time-series-data-python/

In [0]:
pct_df_mean = pct_df(df_mean)

In [0]:
#Ignore this code; helper functions for binarizing
# list_col_names = list(pct_df_mean.columns.values)
from itertools import repeat
def binarize(x):
    if x< 0:
        x = -1
    elif x>0:
        x=1
    else:
        x = 0
    return x
month_cpi = [x for item in CPI_list[1:len(CPI_list)-1] for x in repeat(item, 12)]
month_cpi.insert(len(a),CPI_list[len(CPI_list)-1])
def bin_action_cpi(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action(df):
    df = df.applymap(binarize)
    return df
def binarize10(x):
    if x<= 0:
        x = 0
    else:
        x = 1
    return x
def bin_action_cpi10(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize10)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action10(df):
    df = df.applymap(binarize10)
    return df
def binarize123(x):
    if x< 0:
        x = 1
    elif x>0:
        x=3
    else:
        x = 2
    return x
def bin_action_cpi123(df):
    df.drop(["CPI"], axis=1)
    df = df.applymap(binarize123)
    try:
        df['CPI']=CPI_list[1:]
    except:
        df['CPI']= month_cpi
    return df
def bin_action123(df):
    df = df.applymap(binarize123)
    return df

In [0]:
# percentage change df and binarize 123
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi123(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action123(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize 0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
# df_latest = bin_action_cpi10(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
# df_latest_wo_cpi = bin_action10(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df and binarize -1,0,1
# df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(pct_df_mean,'Y','linear')
# df_latest = bin_action_cpi(df_latest)
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi = bin_action(df_latest_wo_cpi)
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# percentage change df
df_mean_wo_cpi = pct_df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
df_latest = UpsampleByTimeIndex(pct_df_mean,'M','linear')
df_latest.index = list(range(len(df_latest)))
df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'M','linear')
df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

# mean df
# df_mean_wo_cpi = df_mean.drop(["CPI"], axis=1) # remove cpi to create array of test data
# df_latest = UpsampleByTimeIndex(df_mean,'Y','linear')
# df_latest.index = list(range(len(df_latest)))
# df_latest_wo_cpi = UpsampleByTimeIndex(df_mean_wo_cpi,'Y','linear')
# df_latest_wo_cpi.index = list(range(len(df_latest_wo_cpi)))

In [13]:
df_latest_wo_cpi

,ISR_TM_TAX_MANF_BR_ZS,ISR_TM_TAX_MRCH_BR_ZS,ISR_FM_LBL_BMNY_CN,ISR_FM_LBL_BMNY_GD_ZS,ISR_FM_LBL_BMNY_ZG,ISR_FM_LBL_BMNY_IR_ZS,ISR_GC_BAL_CASH_CN,ISR_GC_BAL_CASH_GD_ZS,ISR_AG_PRD_CREL_MT,ISR_AG_YLD_CREL_KG,ISR_NE_GDI_STKB_CN,ISR_NE_GDI_STKB_CD,ISR_BN_RES_INCL_CD,ISR_SE_PRM_UNER_FE,ISR_SE_PRM_UNER,ISR_FS_AST_CGOV_GD_ZS,ISR_SE_PRM_UNER_MA,ISR_FM_AST_CGOV_ZG_M3,ISR_FM_AST_PRVT_ZG_M3,ISR_FS_AST_DOMO_GD_ZS,ISR_FM_AST_DOMO_ZG_M3,ISR_EN_ATM_CO2E_PP_GD_KD,ISR_EN_ATM_CO2E_PP_GD,ISR_EN_ATM_CO2E_KD_GD,ISR_EN_ATM_CO2E_KT,ISR_EN_CO2_ETOT_ZS,ISR_EN_ATM_CO2E_PC,ISR_EN_ATM_CO2E_GF_ZS,ISR_EN_ATM_CO2E_GF_KT,ISR_EN_CO2_MANF_ZS,ISR_EN_ATM_CO2E_LF_ZS,ISR_EN_ATM_CO2E_LF_KT,ISR_EN_CO2_OTHX_ZS,ISR_EN_CO2_BLDG_ZS,ISR_EN_ATM_CO2E_SF_ZS,ISR_EN_CO2_TRAN_ZS,ISR_EN_ATM_CO2E_EG_ZS,ISR_EN_ATM_CO2E_SF_KT,ISR_EG_USE_CRNW_ZS,ISR_NY_GDP_COAL_RT_ZS,...,ISR_NY_ADJ_DFOR_GN_ZS,ISR_NY_ADJ_NNAT_GN_ZS,ISR_NY_ADJ_NNAT_CD,ISR_NY_ADJ_DFOR_CD,ISR_NY_ADJ_DPEM_GN_ZS,ISR_SP_ADO_TFRT,ISR_NY_ADJ_DPEM_CD,ISR_SP_POP_DPND_OL,ISR_SP_POP_DPND,ISR_SP_POP_DPND_YG,ISR_AG_LND_AGRI_K2,ISR_AG_LND_AGRI_ZS,ISR_TX_VAL_AGRI_ZS_UN,ISR_TM_VAL_AGRI_ZS_UN,ISR_NV_AGR_TOTL_ZS,ISR_NV_AGR_TOTL_KD_ZG,ISR_EA_PRD_AGRI_KD,ISR_NV_AGR_TOTL_CN,ISR_NV_AGR_TOTL_KD,ISR_NV_AGR_TOTL_KN,ISR_NV_AGR_TOTL_CD,ISR_IS_AIR_GOOD_MT_K1,ISR_IS_AIR_PSGR,ISR_IS_AIR_DPRT,ISR_EG_USE_COMM_CL_ZS,ISR_AG_LND_ARBL_HA,ISR_AG_LND_ARBL_HA_PC,ISR_AG_LND_ARBL_ZS,ISR_MS_MIL_TOTL_P1,ISR_MS_MIL_TOTL_TF_ZS,ISR_MS_MIL_XPRT_KD,ISR_MS_MIL_MPRT_KD,ISR_FB_BNK_CAPA_ZS,ISR_FD_RES_LIQU_AS_ZS,ISR_FB_AST_NPER_ZS,ISR_TM_TAX_MANF_BC_ZS,ISR_TM_TAX_MRCH_BC_ZS,ISR_TM_TAX_TCOM_BC_ZS,ISR_SP_DYN_CBRT_IN,ISR_TM_TAX_TCOM_BR_ZS
0,0.000000,0.000000,0.080206,-0.028810,-0.483826,0.064877,0.688141,0.517404,0.486385,0.464457,0.354108,0.374832,-0.106008,-0.088312,-0.093993,-1.073336,-0.100495,-4.063186,-0.142224,0.000000,0.000000,0.000261,-0.044739,0.000261,0.081973,0.004731,0.053758,-0.075762,0.000000,-0.117354,-0.046640,0.031509,-0.033832,0.137691,0.076715,-0.014072,-0.005006,0.164977,-0.080389,0.0,...,-1.000000,-0.023495,0.081130,-1.000000,-0.031944,-0.018821,0.071777,0.004715,-0.003866,-0.006890,0.012522,0.012522,-0.378179,-0.091183,-0.117522,-2.330366,0.097389,-0.018463,0.079689,0.079689,-0.003442,-0.143668,0.101598,0.093547,0.017173,0.015015,-0.011454,0.015015,0.005291,-0.030829,1.126316,-0.691016,0.137285,0.000000,-0.233333,0.000000,0.000000,0.000000,0.009302,0.000000
1,0.000310,0.000107,0.080698,-0.020900,-0.437381,0.063517,0.679450,0.520521,0.472635,0.426144,0.341844,0.357770,-0.182495,-0.106910,-0.103239,-2.730595,-0.098900,-3.932049,-0.158783,-0.001256,0.000000,0.004832,-0.038369,0.004832,0.079756,0.005315,0.051844,-0.073821,0.000000,-0.118352,-0.043185,0.033042,-0.007953,0.115639,0.072618,-0.015364,-0.004150,0.158266,-0.077647,0.0,...,-0.916665,-0.033240,0.063245,1991.681639,-0.028873,-0.018851,0.066879,0.004287,-0.003650,-0.006447,0.010743,0.010743,-0.353453,-0.088114,-0.094119,-2.130249,0.097893,-0.001496,0.080161,0.080161,0.009258,-0.134344,0.084207,0.072786,0.015613,0.014750,-0.011477,0.014750,0.000695,-0.034273,1.045864,-0.687100,0.130411,-0.005807,-0.198188,0.000000,0.000000,0.000000,0.006607,-0.000017
2,0.000621,0.000213,0.081189,-0.012991,-0.390935,0.062157,0.670758,0.523638,0.458884,0.387832,0.329581,0.340709,-0.258983,-0.125508,-0.112484,-4.387854,-0.097305,-3.800912,-0.175342,-0.002511,0.000000,0.009402,-0.032000,0.009402,0.077540,0.005898,0.049930,-0.071880,0.000000,-0.119350,-0.039731,0.034575,0.017925,0.093588,0.068521,-0.016655,-0.003294,0.151555,-0.074904,0.0,...,-0.833330,-0.042984,0.045359,3984.363278,-0.025803,-0.018882,0.061981,0.003859,-0.003434,-0.006004,0.008963,0.008963,-0.328726,-0.085044,-0.070716,-1.930132,0.098398,0.015471,0.080633,0.080633,0.021958,-0.125019,0.066816,0.052025,0.014052,0.014485,-0.011501,0.014485,-0.003901,-0.037716,0.965412,-0.683184,0.123537,-0.011614,-0.163043,0.000000,0.000000,0.000000,0.003912,-0.000033
3,0.000931,0.000320,0.081681,-0.005082,-0.344490,0.060797,0.662067,0.526756,0.445133,0.349519,0.317317,0

In [14]:
# check for null, if false then no missing values
def null_check(dfr):
    return dfr.isnull().values.any()
# print(null_check(df0))
# print(null_check(df_mean))
print(null_check(df_latest_wo_cpi))

False


In [15]:
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))

input_data = df_latest.values
test_data = df_latest_wo_cpi.values
input_data[:,0:len(input_data[0])-1] = sc.fit_transform(input_data[:,0:len(input_data[0])-1])
test_data[:,0:len(test_data[0])] = sc.fit_transform(test_data[:,:])
test_data

array([[0.97749688, 0.62751181, 0.80185419, ..., 0.85158643, 0.69649685,
        0.12166918],
       [0.97937214, 0.62873899, 0.80323911, ..., 0.85158643, 0.63922169,
        0.12155256],
       [0.9812474 , 0.62996616, 0.80462402, ..., 0.85158643, 0.58194653,
        0.12143595],
       ...,
       [0.86050392, 0.62667623, 0.73177015, ..., 0.79773484, 0.38274072,
        0.26805765],
       [0.9190004 , 0.62709402, 0.73548489, ..., 0.82466063, 0.39089977,
        0.19486341],
       [0.97749688, 0.62751181, 0.73919963, ..., 0.85158643, 0.39905882,
        0.12166918]])

In [16]:
print(test_data.shape)
print(input_data.shape) # check data and shape, are there 22/ 229 rows?

(193, 662)
(193, 663)


In [17]:
df_latest.head()

,ISR_TM_TAX_MANF_BR_ZS,ISR_TM_TAX_MRCH_BR_ZS,ISR_FM_LBL_BMNY_CN,ISR_FM_LBL_BMNY_GD_ZS,ISR_FM_LBL_BMNY_ZG,ISR_FM_LBL_BMNY_IR_ZS,ISR_GC_BAL_CASH_CN,ISR_GC_BAL_CASH_GD_ZS,ISR_AG_PRD_CREL_MT,ISR_AG_YLD_CREL_KG,ISR_NE_GDI_STKB_CN,ISR_NE_GDI_STKB_CD,ISR_BN_RES_INCL_CD,ISR_SE_PRM_UNER_FE,ISR_SE_PRM_UNER,ISR_FS_AST_CGOV_GD_ZS,ISR_SE_PRM_UNER_MA,ISR_FM_AST_CGOV_ZG_M3,ISR_FM_AST_PRVT_ZG_M3,ISR_FS_AST_DOMO_GD_ZS,ISR_FM_AST_DOMO_ZG_M3,ISR_EN_ATM_CO2E_PP_GD_KD,ISR_EN_ATM_CO2E_PP_GD,ISR_EN_ATM_CO2E_KD_GD,ISR_EN_ATM_CO2E_KT,ISR_EN_CO2_ETOT_ZS,ISR_EN_ATM_CO2E_PC,ISR_EN_ATM_CO2E_GF_ZS,ISR_EN_ATM_CO2E_GF_KT,ISR_EN_CO2_MANF_ZS,ISR_EN_ATM_CO2E_LF_ZS,ISR_EN_ATM_CO2E_LF_KT,ISR_EN_CO2_OTHX_ZS,ISR_EN_CO2_BLDG_ZS,ISR_EN_ATM_CO2E_SF_ZS,ISR_EN_CO2_TRAN_ZS,ISR_EN_ATM_CO2E_EG_ZS,ISR_EN_ATM_CO2E_SF_KT,ISR_EG_USE_CRNW_ZS,ISR_NY_GDP_COAL_RT_ZS,...,ISR_NY_ADJ_NNAT_GN_ZS,ISR_NY_ADJ_NNAT_CD,ISR_NY_ADJ_DFOR_CD,ISR_NY_ADJ_DPEM_GN_ZS,ISR_SP_ADO_TFRT,ISR_NY_ADJ_DPEM_CD,ISR_SP_POP_DPND_OL,ISR_SP_POP_DPND,ISR_SP_POP_DPND_YG,ISR_AG_LND_AGRI_K2,ISR_AG_LND_AGRI_ZS,ISR_TX_VAL_AGRI_ZS_UN,ISR_TM_VAL_AGRI_ZS_UN,ISR_NV_AGR_TOTL_ZS,ISR_NV_AGR_TOTL_KD_ZG,ISR_EA_PRD_AGRI_KD,ISR_NV_AGR_TOTL_CN,ISR_NV_AGR_TOTL_KD,ISR_NV_AGR_TOTL_KN,ISR_NV_AGR_TOTL_CD,ISR_IS_AIR_GOOD_MT_K1,ISR_IS_AIR_PSGR,ISR_IS_AIR_DPRT,ISR_EG_USE_COMM_CL_ZS,ISR_AG_LND_ARBL_HA,ISR_AG_LND_ARBL_HA_PC,ISR_AG_LND_ARBL_ZS,ISR_MS_MIL_TOTL_P1,ISR_MS_MIL_TOTL_TF_ZS,ISR_MS_MIL_XPRT_KD,ISR_MS_MIL_MPRT_KD,ISR_FB_BNK_CAPA_ZS,ISR_FD_RES_LIQU_AS_ZS,ISR_FB_AST_NPER_ZS,ISR_TM_TAX_MANF_BC_ZS,ISR_TM_TAX_MRCH_BC_ZS,ISR_TM_TAX_TCOM_BC_ZS,ISR_SP_DYN_CBRT_IN,ISR_TM_TAX_TCOM_BR_ZS,CPI
0,0.977497,0.627512,0.801854,0.704531,0.986579,0.972991,0.503741,0.467761,1.000000,1.000000,0.872633,0.883627,0.561012,0.402541,0.247353,0.886155,0.178123,0.438890,0.895160,0.471345,0.898824,0.325489,0.227450,0.325489,0.928532,0.494315,0.684978,0.003798,0.004518,0.281211,0.368345,0.608912,0.100275,0.295630,0.633657,0.373142,0.408244,1.000000,0.282726,0.0,...,0.449998,0.481717,0.000000,0.380412,1.000000,0.455579,0.172594,0.109178,0.104319,0.793893,0.793893,0.153338,0.076600,0.192078,0.997133,0.644791,0.317687,0.615584,0.615584,0.439443,0.080728,0.465938,0.636239,0.451024,0.697252,0.632158,0.697252,0.546076,0.434790,1.000000,0.019304,1.000000,0.191667,0.220518,0.479597,0.501725,0.851586,0.696497,0.121669,66.000000
1,0.979372,0.628739,0.803239,0.729011,0.987001,0.969580,0.499983,0.469177,0.982481,0.941232,0.871730,0.882271,0.559802,0.368996,0.226740,0.812309,0.181299,0.442106,0.894582,0.460678,0.898824,0.335568,0.237356,0.335568,0.917923,0.499584,0.677782,0.003810,0.004518,0.280632,0.375265,0.611593,0.103363,0.288025,0.618812,0.366918,0.413757,0.980608,0.287152,0.0,...,0.435221,0.462351,0.083333,0.411636,0.999479,0.440542,0.159677,0.120065,0.128199,0.776359,0.776359,0.171331,0.090601,0.256102,0.997297,0.646933,0.362408,0.617588,0.617588,0.467689,0.093222,0.428702,0.599184,0.447937,0.695049,0.631955,0.695049,0.500570,0.398558,0.945815,0.020219,0.985292,0.188744,0.236869,0.479597,0.501725,0.851586,0.639222,0.121553,66.833333
2,0.981247,0.629966,0.804624,0.753492,0.987422,0.966168,0.496224,0.470592,0.964962,0.882464,0.870827,0.880914,0.558592,0.335451,0.206128,0.738462,0.184475,0.445322,0.894004,0.450011,0.898824,0.345647,0.247263,0.345647,0.907314,0.504854,0.670586,0.003822,0.004518,0.280053,0.382185,0.614274,0.106451,0.280420,0.603967,0.360695,0.419270,0.961217,0.291578,0.0,...,0.420445,0.442985,0.166667,0.442860,0.998957,0.425506,0.146760,0.130952,0.152079,0.758825,0.758825,0.189324,0.104602,0.320126,0.997460,0.649075,0.407129,0.619592,0.619592,0.495936,0.105715,0.391465,0.562129,0.444851,0.692846,0.631753,0.692846,0.455063,0.362325,0.891630,0.021133,0.970584,0.185822,0.253220,0.479597,0.501725,0.851586,0.581947,0.121436,67.666667
3,0.983123,0.631193,0.806009,0.777973,0.987843,0.962756,0.492466,0.472007,0.947443,0.823696,0.869924,0.879557,0.557381,0.301906,0.185515,0.664616,0.187651,0.448538,0.893425,0.439344,0.898824,0.355726,0.257170,0.355726,0.896705,0.51

In [19]:
n_steps = 3
df_test=test_data[-n_steps:]
CPI_test = df_latest['CPI'][-n_steps:]
print(CPI_test)
print(df_test.shape)
input_feature = input_data[:len(input_data)-n_steps] # to remove last 5 rows, follow the time step you set

190    63.50
191    63.75
192    64.00
Name: CPI, dtype: float64
(3, 662)


In [20]:
input_feature.shape # check shape 

(190, 663)

In [0]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [22]:
# choose a number of time steps
from numpy import array
# convert into input/output
X, y = split_sequences(input_feature, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])
# the array is (Total number of samples, time step, total number of dimensions)

(188, 3, 662) (188,)
[[0.97749688 0.62751181 0.80185419 ... 0.85158643 0.69649685 0.12166918]
 [0.97937214 0.62873899 0.80323911 ... 0.85158643 0.63922169 0.12155256]
 [0.9812474  0.62996616 0.80462402 ... 0.85158643 0.58194653 0.12143595]] 67.66666666666667
[[0.97937214 0.62873899 0.80323911 ... 0.85158643 0.63922169 0.12155256]
 [0.9812474  0.62996616 0.80462402 ... 0.85158643 0.58194653 0.12143595]
 [0.98312266 0.63119334 0.80600893 ... 0.85158643 0.52467137 0.12131934]] 68.5
[[0.9812474  0.62996616 0.80462402 ... 0.85158643 0.58194653 0.12143595]
 [0.98312266 0.63119334 0.80600893 ... 0.85158643 0.52467137 0.12131934]
 [0.98499792 0.63242052 0.80739385 ... 0.85158643 0.4673962  0.12120273]] 69.33333333333333
[[0.98312266 0.63119334 0.80600893 ... 0.85158643 0.52467137 0.12131934]
 [0.98499792 0.63242052 0.80739385 ... 0.85158643 0.4673962  0.12120273]
 [0.98687318 0.63364769 0.80877876 ... 0.85158643 0.41012104 0.12108612]] 70.16666666666667
[[0.98499792 0.63242052 0.80739385 ... 0

In [23]:
# final model
# https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/ 
# refer to this if necessary
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
model = Sequential()
model.add(LSTM(75, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# model.save_weights('my_model_weights.h5')

Using TensorFlow backend.


In [24]:
# fit model
model.fit(X, y, epochs=200, verbose=0)#, validation_split=0.2, batch_size=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [0]:
# demonstrate prediction
x_input = df_test
x_input = x_input.reshape((1, n_steps,X.shape[2] ))
yhat = model.predict(x_input, verbose=0)

Percentage Error

In [26]:
print(yhat)
actual = CPI_test[len(df_latest)-1]
print(float((abs(actual-yhat)/actual)*100))

[[64.11757]]
0.1837015151977539


In [27]:
weight_tensors = model.layers[1].weights
weight_names = list(map(lambda x: x.name, weight_tensors))
print(weight_names)

['dense_1/kernel:0', 'dense_1/bias:0']


In [28]:
for layer in model.layers:
        if "LSTM" in str(layer):
            weightLSTM = layer.get_weights()
warr,uarr, barr = weightLSTM
print(warr.shape) # warr is a numpy array of weights for inputs
print(uarr.shape) # uarr is a numpy array of weights for hidden units
print(barr.shape) # barr is a numpy array of bias

(662, 300)
(75, 300)
(300,)


In [29]:
print(warr)
print(uarr)
print(barr)

[[ 0.07012057 -0.06335633  0.0359504  ...  0.08020607  0.0062677
   0.09002765]
 [-0.04778994 -0.01383412  0.02498766 ... -0.04969701  0.05096054
   0.08433254]
 [-0.05270842 -0.07622582 -0.07143743 ...  0.03624085 -0.05663345
   0.06516201]
 ...
 [ 0.01686111 -0.03424168 -0.01182953 ...  0.08287866  0.05577856
   0.04908691]
 [ 0.01304785  0.03902033  0.03800482 ... -0.03828614  0.02034367
  -0.02387229]
 [ 0.08308981 -0.01035748  0.05303033 ... -0.02054707  0.06425335
   0.05678686]]
[[ 0.03278714 -0.01863568  0.00600876 ...  0.11719739 -0.06404472
   0.0542938 ]
 [-0.00505089  0.11117744  0.04114025 ... -0.1373352  -0.05849872
   0.04638517]
 [ 0.06346167 -0.08154555  0.00836413 ... -0.04769276  0.01588806
  -0.05959089]
 ...
 [ 0.06867906  0.02104246 -0.08039384 ...  0.03915113 -0.0378398
  -0.01930517]
 [ 0.06888973  0.07093787 -0.01846262 ...  0.0541907  -0.17388041
  -0.0506287 ]
 [ 0.11402974  0.12861237 -0.00527957 ...  0.08538972  0.0500888
   0.02509863]]
[ 1.52205313e-02 -7

In [0]:
# change df to percentage changes
# play with the rank/ totaln
# interpolate lesser 
# time step

In [0]:
time step = 5, CPI, indicator data
interpolate by year, 22 data points
# 20.441774368286133
interpolate by month, 262 DP
# 1.0591048002243042
interpolate by day, 7920 DP
# 0.015229816548526287 

After performing percentage change and normalizing 
interpolate by year time step 7
# 0.6501209735870361
# 351430.6875
interpolate by year time step 5
# 0.5464288592338562
# 320606.21875
interpolate by year time step 3
# 1.6997615098953247
interpolate by month time step 7
# 1.4450266361236572
interpolate by month time step 5
# 0.217756450176239


*** champion ***
interpolate by month time step 3
# 0.4332916736602783 25 units
# 0.15020249783992767 50 units 0.24396678805351257
# 0.08212704956531525 75 units
# 0.025785423815250397 75 units dense 15 , 1
# 0.09989678114652634 75 units dense 25 , 1
***  winner  # 0.016958503052592278 75 units dense 35 , 1 
*** end ***


binarize -1,0,1
year time step = 7
# 63.047935485839844
year time step = 5
# 59.16572189331055
year time step = 3
# 82.13848114013672
month time step = 7
# 11.621817588806152
month time step = 5
# 11.747268676757812
month time step = 3
# 15.268765449523926


binarize 0,1
year time step = 7
# 1.3483071327209473
year time step = 5
# 2.037656784057617
year time step = 3
# 4.179565906524658
month time step = 7
# 0.44504478573799133
month time step = 5
# 4.9152421951293945
month time step = 3
# 3.9173173904418945

binarize 1,2,3
year time step = 7
# 5.180117607116699
year time step = 5
# 4.566781520843506
year time step = 3
# 1.8392828702926636
month time step = 7
# 3.652085065841675
month time step = 5
# 4.432069778442383
month time step = 3
# 0.7650640606880188